# Streamlining the Process of Exploring Datasets

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Datasets with format #1: 

**Shortened-section-name_data_Shortened-discipline-name_table-number**

***E.g. Satis_data_Arts_1***

## Importing Datasets

In [2]:
#can run this if working in the same folder as the data
def read_data(survey_section, discipline, dataset):
    ''' Loads UCUES datasets present in the Data Mural Project GitHub Data Folder and returns the dataset as a pandas
    dataframe
    
    Params:
        survey_section  str, Shortened section name, ex: Satis 
        discipline    str, Shortened discipline name, ex: Hum
        dataset    int, the sub-dataset from the specified survey section 
    '''
    #Creates the path for where the file is located in the local environment
    file_path = "{}_data_{}_{}.csv".format(survey_section, discipline, dataset)
    DF = pd.read_csv(file_path, encoding='utf-16le', sep = '\t')
    return DF

In [3]:
def read_data_from_file_path(file_path, UCUES_dataset, survey_section, discipline, dataset):
    ''' Loads UCUES datasets present in the Data Mural Project GitHub Data Folder and returns the dataset as a pandas
    dataframe
    
    Params:
        file_path     str, file_path to the dataset on your local environment, ex: /Users/omarramos/Documents/Data_Mural_Project/Data
        UCUES_dataset   str, name of the UCUES survey section and sub-dataset number, ex: Satisfaction-1
        survey_section  str, Shortened section name, ex: Satis 
        discipline    str, Shortened discipline name, ex: Hum
        dataset    int, the sub-dataset from the specified survey section 
    '''
    #Creates the path for where the file is located in the local environment
    file_path = "{}/Data/UCUES-{}/{}_data_{}_{}.csv".format(file_path, UCUES_dataset,survey_section, discipline, dataset)
    DF = pd.read_csv(file_path, encoding='utf-16le', sep = '\t')
    return DF

## Open and merge datasets

In [4]:
# example for how to use the function to open datasets for different disciplines for the same section
# and for how to create a new dataframe with all the disciplines and their scores merged together
def create_dataset(file_path, UCUES_dataset, survey_section, discipline, dataset):
    disciplines = ["Arts", "Hum", "Life", "Eng", "Health", "Phys", "Prof", "Social", "Undec"]
    dataset = dataset
    survey_section = survey_section
    file_path = file_path
    UCUES_dataset = UCUES_dataset
    
    #make array with the different datasets for each discipline
    array_dfs = []
    for discipline in disciplines:
        df = read_data_from_file_path(file_path, UCUES_dataset, survey_section, discipline, dataset)
        df[discipline] = [discipline for i in range(len(df))]
        array_dfs.append(df)
        
    #merge datasets for each discipline
    concat_df = pd.concat(array_dfs)
    
    #one-hot encode discipline columns
    concat_df = concat_df.fillna(0) #convert NaNs to 0
    for discipline in disciplines:
        concat_df.loc[concat_df[discipline] == discipline, discipline] = 1
        
    # Get one hot encoding of column with scores
    one_hot = pd.get_dummies(concat_df['Pivot Field Values'])
    # Drop column with scores as it is now encoded
    one_hot_df = concat_df.drop('Pivot Field Values',axis = 1)
    # Join the encoded df
    one_hot_df = one_hot_df.join(one_hot)
    
    #rename columns
    one_hot_df = one_hot_df.rename(columns = {"Label1": "Statement", "Calculation1": "Percent_pop", 
                                              "Total": "Pop_raw_count"})
    
    #drop duplicate rows
    new_df = one_hot_df.drop_duplicates()
    #reset index
    new_df = new_df.reset_index()
    
    #convert string percent column to actual percentages
    for x in range(len(new_df["Percent_pop"])):
        new_df["Percent_pop"][x] = float(new_df["Percent_pop"][x].replace("%", ""))/100
    
    return new_df
    

## Exploratory Data Analysis

In [7]:
#how many people across all discplines relate to question on likert scale
def plot_total_scores(new_df, likert_scale):
    #get numeric columns, aka the scores
    cols = get_unique_scores(new_df)
    unique_stats = np.unique(new_df["Statement"])

    for stat in unique_stats:
        stat1_df = new_df.where(new_df["Statement"] == stat).dropna()
        stat1_all = stat1_df.groupby(cols).sum().drop(columns = "index") 
        stat1_all = stat1_all["Pop_raw_count"].reset_index().drop(columns = cols)
        stat1_all["Score"] = pd.get_dummies(cols[::-1]).idxmax(1)

        plt.figure(figsize=(5,5))
        plt.bar(stat1_all["Score"], stat1_all["Pop_raw_count"])
        plt.xlabel("Score")
        plt.ylabel("Population Raw Count")
        plt.title(stat)
        plt.xticks(cols, likert_scale, rotation=50);

In [6]:
# plotting data for specific disciplines
def plot_disciplines(new_df):
    disciplines = ["Arts", "Hum", "Life", "Eng", "Health", "Phys", "Prof", "Social", "Undec"]
    new_df["Discipline"] = disciplines * 42
    plot_df = new_df[["Statement", "Percent_pop", "Pop_raw_count", 1,2,3,4,5,6, "Discipline"]]
    plot_df
    
    unique_stats = np.unique(plot_df["Statement"])
    
    for i in range(len(unique_stats)):
        plot1 = plot_df.where(plot_df["Statement"] == unique_stats[i]).dropna()
        plot1["Score_value"] = [6]*9 + [5]*9 + [4]*9 + [3]*9 + [2]*9 + [1]*9
        scores = ["Strongly Disagree", "Disagree", "Somewhat Disagree", "Somewhat Agree" ,"Agree" , "Strongly Agree"]
        plot1["Score"] = ["Strongly Agree"] * 9 + ["Agree"]* 9 + ["Somewhat Agree"]* 9 + ["Somewhat Disagree"]* 9 + ["Disagree"] * 9 + ["Strongly Disagree"]* 9

        plt.figure(figsize = (10,8))
        sns.barplot(data = plot1, x = "Discipline", y = "Pop_raw_count", hue = "Score", palette= "flare");
        plt.title(unique_stats[i]);
        plt.ylabel("Population Raw Count");

# Datasets with format #1: 

**Shortened-section-name_data_Shortened-discipline-name_table-number**

***E.g. Satis_data_Arts_1***